# E-Commerce Recommendations

<b> Author:</b> Dominick DiTucci <br>
<b>email:</b> djditucci@gmail.com<br>
<br>
The purpose of this project is to identify reasonable complements to items a customer is interested in by identifying other items that are often purchased in the same transaction as the item of interest. The reasonable complements can be recommended to the customer through the context of "customers who viewed ... also viewed..." and "customers who purchased ... also purchased..." for the purpose of identifying alternative items or add-on items.

The recommendations will be generated by using the apriori algorithm to extract frequent itemsets for association rule mining. The association rules will take the antecedent (item being viewed/purchased) and recommend associated consequents (complementary items) with the specified support and lift.

####  <u> Key Terms </u>
<ul><b>Association Rules:</b> Probability of association between items based on how frequently they are purchased together. If (X) purchased then (Y) has a higher probability of being purchased. </ul>
<ul><b>Antecedent:</b> Item being viewed or purchased. </ul> 
<ul><b>Consequent:</b> Other item(s) that are frequently purchased in the same transaction.</ul>
<ul><b>Support:</b> The relative frequency the antecedent and consequent show up in the same transactions.</ul>
<ul><b>Lift:</b> The ratio of the observed support to what is expected if the two rules were independent and the item association was completely random.</ul>
<ul><b>Confidence:</b> The reliability measure for the association rules.</ul>
<br>
The data used will be two datasets that have been combined. There is a dataset with six months of transactions where items in the "adhesives and sealants" category were purchased and another dataset containing all transactions over a two week period. The raw data contains:

<ul><b>Order Number:</b> The unique identifier for each individual transaction</ul>
<ul><b>l1:</b> Level 1 product identifier (most broad)</ul>
<ul><b>l2:</b> Level 2 product identifier</ul>
<ul><b>l3:</b> Level 3 product identifier (most granular)</ul>
<ul><b>Sku:</b> The unique identifier for a specific item</ul>
<ul><b>Brand:</b> The unique identifier for the specific brand of an item </ul>

In [1]:
# Import Libraries
import sqlite3
import pandas as pd
import numpy as np
#!pip install mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

### Data Preparation
Load the raw data into pandas dataframes.

In [11]:
# Load txt files into pandas and separate columns
adhesiveSealantTrans = pd.read_csv("/Users/DJ/E-Commerce-Recommendations/e-comm-data/Transactions with A&S.txt",sep='\t', header=0)
allTransaction = pd.read_csv("/Users/DJ/E-Commerce-Recommendations/e-comm-data/All Transations - 2 Weeks.txt", sep='\t', header=0)

In [12]:
# Verify the data loaded correctly
adhesiveSealantTrans.head(5)

,order_number,l1,l2,l3,sku,brand
0,1182478,"Adhesives, Sealants and Tape",Putties,Putties,122821,2180
1,63908,"Adhesives, Sealants and Tape",Caulks,Caulks and Sealants,276172,1054
2,1165840,"Adhesives, Sealants and Tape",Concrete and Asphalt,Concrete Cleaners and Sealers,110119,4326
3,928748,Material Handling,Casters and Wheels,Plate Casters,71052,1793
4,1088891,"Adhesives, Sealants and Tape",Glues and Cements,Retaining Compounds,251285,2515


In [13]:
# Verify the data loaded correctly
allTransaction.head(5)

,order_number,l1,l2,l3,sku,brand
0,168266,Power Tools,Power Saws and Accessories,Reciprocating Saw Blades,265105,2768
1,123986,Safety,Spill Control Supplies,Temporary Leak Repair,215839,586
2,158978,Hardware,Door Hardware,Thresholds,284756,1793
3,449035,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,12579,1231
4,781232,Motors,General Purpose AC Motors,General Purpose AC Motors,194681,2603


<br>
Both dataframes will be combined since the goal is recommending the items most frequently associated with each other regardless of category.

In [16]:
# Combine transaction data since adhesives/sealants do not need to be seperate from all other transactions
frames = [adhesiveSealantTrans, allTransaction]
allTransactions = pd.concat(frames)
allTransactions.head()

,order_number,l1,l2,l3,sku,brand
0,1182478,"Adhesives, Sealants and Tape",Putties,Putties,122821,2180
1,63908,"Adhesives, Sealants and Tape",Caulks,Caulks and Sealants,276172,1054
2,1165840,"Adhesives, Sealants and Tape",Concrete and Asphalt,Concrete Cleaners and Sealers,110119,4326
3,928748,Material Handling,Casters and Wheels,Plate Casters,71052,1793
4,1088891,"Adhesives, Sealants and Tape",Glues and Cements,Retaining Compounds,251285,2515


## Exploratory  Analysis

### SQL Queries

Brief analysis will be done using SQL queries. First, the database will be created. Then the database will be connected and loaded with the combined data.

In [10]:
# Create database
db = sqlite3.connect("transactions.db")

# Connect to database
con = sqlite3.connect("/Users/DJ/E-Commerce-Recommendations/transactions.db")

In [17]:
# Move dataframe to sql table in transactions.db
allTransactions.to_sql("all_trans", con, if_exists="replace", index=False)

Let's verify there are no missing values.

In [18]:
# Check all_trans for missing data
print("\nAll Transactions Missing Data")
print("Brand", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN brand is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))
print("Sku", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN sku is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))
print("L1", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l1 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))
print("L2", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l2 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))
print("L3", pd.read_sql_query("SELECT CAST(SUM(CASE WHEN l3 is NULL THEN 1 ELSE 0 END) as float)/COUNT(*) as ProportionMissing FROM all_trans;", con))



All Transactions Missing Data
Brand    ProportionMissing
0                0.0
Sku    ProportionMissing
0                0.0
L1    ProportionMissing
0                0.0
L2    ProportionMissing
0                0.0
L3    ProportionMissing
0                0.0


There is no missing data.

What are the 10 most-purchased items?

In [19]:
# See top 10 items in ALL transactons
pd.read_sql_query("SELECT l1,l2, l3, COUNT(sku) FROM all_trans GROUP BY sku ORDER BY COUNT(sku) DESC LIMIT 10;", con)

,l1,l2,l3,COUNT(sku)
0,"Adhesives, Sealants and Tape",Tapes,Electrical Tapes,35812
1,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,30079
2,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,25550
3,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,10981
4,"Adhesives, Sealants and Tape",Thread and Gasket Sealants,Pipe Sealant Tape,10920
5,"Adhesives, Sealants and Tape",Tapes,Duct and Cloth Tapes,10162
6,"Adhesives, Sealants and Tape",Tapes,Duct and Cloth Tapes,8549
7,"Electronics, Appliances, and Batteries",Batteries,Standard Batteries,8468
8,Cleaning,Paper Products and Dispensers,"Paper Towels, Rolls",8288
9,"Adhesives, Sealants and Tape",Thread and Gasket Sealants,Pipe Sealant Tape,8229


Electrical tape is the most-purchased item with almost 36,000 units in the transaction data. Batteries are the next most-purchased items, followed by pipe sealant tape.

### Pandas Exploration
Pandas will be used to check data types and number of unique values.

In [20]:
# Check data types
print("All Transactions Data", allTransactions.dtypes)

All Transactions Data order_number     int64
l1              object
l2              object
l3              object
sku              int64
brand            int64
dtype: object


In [21]:
# See unique values for all features
print("All Transactions Data\n", allTransactions.nunique())

All Transactions Data order_number    1243048
l1                   33
l2                  593
l3                 6355
sku              333127
brand              4841
dtype: int64


## Building a recommendation system

Given the limited amount of data available, we will find frequent itemsets using the apriori algorithm to find which items are purchased together more often than expected. All items will be grouped by their order number to identify what was purchased in the same transaction. We will then use the most-granular (l3) product hierarchy to find what each item is and make recommendations based on that. The broader product hierarchy levels will be dropped along with the numerical unique identifiers (brand and sku) as they are not useful for our purposes. 

### Prepare Data

In [31]:
# Check shape before filtering rows
allTransactions.shape

(5494816, 6)

Sku and brand will be dropped since we do not have information on which items and brands these numbers correspond to. The l1 and l2 categories will also be dropped because we need the most granular (l3) data available to make meaningful recommendations.

In [32]:
# Drop sku and brand 
allTransactions.drop('sku', axis=1, inplace=True)
allTransactions.drop('brand', axis=1, inplace=True)

# Drop most broad categories and keep most granular
allTransactions.drop('l1', axis=1, inplace=True)
allTransactions.drop('l2', axis=1, inplace=True)

Now the data will be grouped by order number so all items purchased in the same transaction are in a list together.

In [33]:
# Combine items into lists by order number
grouped = allTransactions.groupby('order_number')['l3'].apply(list)
grouped.head()

order_number
1                     [Steel-Toe Work Boots and Shoes]
2    [Bib Overalls, Traffic Safety Vests, Work Cove...
3                                          [Rainsuits]
4                     [Steel-Toe Work Boots and Shoes]
5                     [Steel-Toe Work Boots and Shoes]
Name: l3, dtype: object

In [34]:
# Check shape
grouped.shape

(1243048,)

There are over 1 million orders. We will filter out orders with less than 10 items to make our data more relevant for finding items that are purchased together.

In [35]:
#Filter out small orders
filtered_group = [x for x in grouped if len(x)>=10 ]

# Sample first few large orders
filtered_group[:3]

[['Bib Overalls',
  'Traffic Safety Vests',
  'Work Coveralls',
  'Bib Overalls',
  'Traffic Safety Vests',
  'Traffic Safety Vests',
  'Bib Overalls',
  'Headbands, Sweatbands and Cooling Headwear',
  'Hard Hats',
  'Hard Hat Accessories',
  'Traffic Warning Flags and Banners',
  'Traffic Warning Flags and Banners',
  'Bib Overalls',
  'Hard Hats',
  'High Visibility Shirts and Sweatshirts'],
 ['Cut-Resistant Gloves',
  'Cut-Resistant Gloves',
  'Leather Work Gloves',
  'Sports Drinks and Thirst Quenchers',
  'Safety Glasses',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Leather Work Gloves',
  'Standard Batteries',
  'Standard Batteries',
  'Standard Batteries',
  'Standard Batteries',
  'Standard Batteries',
  'Standard Batteries',
  'Safety Glasses',
  'Safety Glasses',
  'Safety Glasses',
  'Cut-Resis

Now the data can be one-hot encoded using the MLxtend library

In [36]:
# One-hot encode data in pandas dataframe
te = TransactionEncoder()
te_ary = te.fit(filtered_group).transform(filtered_group)
transaction_group = pd.DataFrame(te_ary, columns=te.columns_)
transaction_group.head()

,12 Volt Accessories,12-Point Flange Head Cap Screws,3-Ring Binder Accessories,3-Ring Binders,3.3 Inch Diameter Motors,3D Printing Materials,4-Way and Box Parallels,4.4 Inch Diameter Motors,5 X 20mm Glass and Ceramic Fuses,50 Hz Motors,...,Wrist Watches,Writing Instrument Accessories,Y Strainers,Yard Hydrants,Yoke Ends,Zone Valve Accessories,Zone Valve Actuators,Zone Valves,Zoning Dampers,pH Meters
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Find itemsets that are in at least 1% of our data:

In [37]:
# Find items with any support using apriori algorithm to show which have any possible relationship
apriori(transaction_group, min_support=0.01, use_colnames=True)

,support,itemsets
0,0.055478,(Abrasive Cut-Off and Chop Wheels)
1,0.020519,(Abrasive Rolls)
2,0.030859,(Absorbent Pads)
3,0.014272,(Adhesive Bandages)
4,0.019328,(Adhesives)
5,0.024291,(Adjustable Wrenches)
6,0.010042,(Aerosol Dusters)
7,0.021390,(Air Guns)
8,0.013432,(American Glass and Ceramic Fuses)
9,0.043100,(Anti-Seize Compounds)


To remove any orders with only 1 item purchased, we'll create a column with number of items in each order and then filter out single-item transactions.

In [39]:
# Find frequent item sets and and number of items in each order
freq_itemsets = apriori(transaction_group, min_support=0.01, use_colnames=True)
freq_itemsets['length'] = freq_itemsets['itemsets'].apply(lambda x : len(x))
freq_itemsets.head()

,support,itemsets,length
0,0.055478,(Abrasive Cut-Off and Chop Wheels),1
1,0.020519,(Abrasive Rolls),1
2,0.030859,(Absorbent Pads),1
3,0.014272,(Adhesive Bandages),1
4,0.019328,(Adhesives),1


In [40]:
#Filter out smaller orders orders
freq_itemsets[ (freq_itemsets['length'] > 1) &
             (freq_itemsets['support'] > 0.02) ]

,support,itemsets,length
302,0.034891,"(Black Pipe Fittings, Black Pipe)",2
315,0.021764,"(Flat Washers, Black Pipe Fittings)",2
330,0.029003,"(Disposable Gloves, Cable Ties)",2
332,0.028087,"(Duct and Cloth Tapes, Cable Ties)",2
334,0.053065,"(Cable Ties, Electrical Tapes)",2
336,0.036494,"(Cable Ties, Flat Washers)",2
338,0.030003,"(Hex Head Cap Screws, Cable Ties)",2
339,0.024398,"(Hex Nuts, Cable Ties)",2
341,0.031103,"(Cable Ties, Jobber Length Drill Bits)",2
348,0.027453,"(Pipe Sealant Tape, Cable Ties)",2


## Create recommendation engine
Now we can generate a dataframe of association rules and their corresponding metrics. We will focus on the support, lift, and confidence of each association.

In [43]:
# Generate association rules to make recommendations usiong lift metric to choose items that are purchased together 
# more often than expected
rules = association_rules(freq_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cable Ties),(Abrasive Cut-Off and Chop Wheels),0.137088,0.055478,0.012081,0.088124,1.588438,0.004475,1.035800
1,(Abrasive Cut-Off and Chop Wheels),(Cable Ties),0.055478,0.137088,0.012081,0.217756,1.588438,0.004475,1.103124
2,(Abrasive Cut-Off and Chop Wheels),(Depressed Center Wheels),0.055478,0.042527,0.013287,0.239504,5.631817,0.010928,1.259012
3,(Depressed Center Wheels),(Abrasive Cut-Off and Chop Wheels),0.042527,0.055478,0.013287,0.312444,5.631817,0.010928,1.373738
4,(Duct and Cloth Tapes),(Abrasive Cut-Off and Chop Wheels),0.171658,0.055478,0.012256,0.071400,1.286988,0.002733,1.017146


Generally, a lift above 1 can infer some association. We will filter out associations that have a lift of at least 6 and confidence of at least 80% to find the most relevant recommendations.

In [49]:
# Filter items and save
saved_recs = rules[ (rules['lift'] >= 6) &
     rules['confidence'] >= 0.8]
saved_recs.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
8,(Abrasive Cut-Off and Chop Wheels),(Flap Discs),0.055478,0.048239,0.017144,0.309016,6.405927,0.014467,1.377399
9,(Flap Discs),(Abrasive Cut-Off and Chop Wheels),0.048239,0.055478,0.017144,0.355390,6.405927,0.014467,1.465261
74,(Black Pipe Fittings),(Black Pipe),0.057670,0.047575,0.034891,0.605005,12.716959,0.032147,2.411236
75,(Black Pipe),(Black Pipe Fittings),0.047575,0.057670,0.034891,0.733387,12.716959,0.032147,3.534447
274,(Linear Fluorescent Lamps),(Compact Fluorescent Lamps (CFL)),0.030882,0.033188,0.012127,0.392681,11.832098,0.011102,1.591934


The final step now that recommendations have been made is to save them as an exportable csv file.

In [48]:
# Save Recs to csv
saved_recs.to_csv('recommendations.csv', index=False)

##  Summary

The apriori algorithm and association rule mining were used to extract frequent itemsets (items purchased together) from the data and generate rules for frequent item patterns. These rules can be used in a recommendation engine in the context of "customers who viewed ... also viewed..." and "customers who purchased ... also purchased..." in order to encourage add-on purchases or encourage past customers to  continue using "The Company" for their e-commerce needs. The results are contained in a CSV file that can be used across platforms for production purposes.